In [1]:
import numpy as np
import numpy.linalg as lia
import pandas as pd
import matplotlib as plt

In [2]:
from sklearn import datasets
digits = datasets.load_digits()

In [3]:
print(len(digits.data))
print(digits.target.size)

1797
1797


In [4]:
from sklearn.datasets import fetch_openml
wine = fetch_openml(name='wine', version=1)

In [5]:
print(len(wine.data))
print(wine.target.size)

178
178


In [6]:
# normalization of wine data via division of each feature by its max value

wine_data_norm = []
for col in wine.data.T:
    col_norm = col/np.amax(col)
    wine_data_norm.append(col_norm)
    
wine.data = np.asarray(wine_data_norm).T

In [7]:
# 5-fold cross validation for digits dataset

digitsTrainingSetSize = int(np.ceil(0.8 * len(digits.data)))
digitsValidationSetSize = int(len(digits.data) - digitsTrainingSetSize)

xDigitsTrainingSets = []
yDigitsTrainingSets = []
xDigitsValidationSets = []
yDigitsValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []

    for index, data in enumerate(digits.data[foldIndex*digitsValidationSetSize:((foldIndex*digitsValidationSetSize)+digitsValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(digits.target[index+(foldIndex*digitsValidationSetSize)])
    
    xTrainingSet = []
    yTrainingSet = []

    for index, data in enumerate(digits.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(digits.target[index])
            
    xDigitsTrainingSets.append(xTrainingSet)
    yDigitsTrainingSets.append(yTrainingSet)
    xDigitsValidationSets.append(xValidationSet)
    yDigitsValidationSets.append(yValidationSet)
    
# 5-fold cross validation for wine dataset

wineTrainingSetSize = int(np.ceil(0.8 * len(wine.data)))
wineValidationSetSize = int(len(wine.data) - wineTrainingSetSize)

xWineTrainingSets = []
yWineTrainingSets = []
xWineValidationSets = []
yWineValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []
    for index, data in enumerate(wine.data[foldIndex*wineValidationSetSize:((foldIndex*wineValidationSetSize)+wineValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(wine.target[index+(foldIndex*wineValidationSetSize)])
    
    xTrainingSet = []
    yTrainingSet = []
    
    for index, data in enumerate(wine.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(wine.target[index])
            
    xWineTrainingSets.append(xTrainingSet)
    yWineTrainingSets.append(yTrainingSet)
    xWineValidationSets.append(xValidationSet)
    yWineValidationSets.append(yValidationSet)

In [8]:
# one-hot encoding of y for digits dataset

numberOfDigitsTargets = 10
numberOfWineTargets = 3

for index, fold in enumerate(yDigitsTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yDigitsValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsValidationSets[index] = encodedFold

# one-hot encoding of y for wine dataset

for index, fold in enumerate(yWineTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yWineValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineValidationSets[index] = encodedFold


In [9]:
def getRandomIndices(arr, batch_size):
    indices = []
    
    if batch_size > len(arr):
        print("Error: batch size larger than size of dataset.")
        return
    
    while batch_size > 0:
        x = np.floor(np.random.random() * len(arr))
        if x not in indices:
            indices.append(int(x))
            batch_size -= 1
    
    return indices

In [10]:
# gradient descent class
 
class GradientDescent:
    
    def __init__(self, batch_size, learning_rate=0.01, momentum=0.9, max_iters=20, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.batch_size = batch_size
        self.max_iters = max_iters
        self.epsilon = epsilon
        self.deltas = [] # delta w{t}
        
    def run(self, gradient_fn, x, y, w):
        # TODO: use epsilon
        grad = np.inf
        t = 1
        while np.linalg.norm(grad) > self.epsilon and t < self.max_iters:
            print(f'Grad descent step {t}')            
            gradients = gradient_fn(x, y, w, self.batch_size)
            for c in range(len(y[0])):
                wc = []
                if(c==0):
                    w[c] = w[c-1] - self.learning_rate * gradients[c]
                else:
                    delta_w = (self.momentum)*(self.deltas[-1]) + (1-self.momentum)*gradients[c]
                    w[c] = w[c-1] - self.learning_rate * delta_w
                if(c != 0):
                    w_prev = self.deltas[-1]
                self.deltas.append(w[c])
                print(f'=============\n{c} : {w[c]}\n')
            t += 1
            
        return w


In [11]:
# logistic regression

class LogisticRegression:
    def __init__(self, add_bias=True):
        self.add_bias = add_bias
        pass
            
    def fit(self, x, y, optimizer):
        # TODO: add bias
        
        def gradient(x, y, w, batch_size):
            gradients = np.zeros(len(w)).tolist()

            indices = getRandomIndices(x, batch_size)

            for index in indices:
                a = np.asarray(x[index])
                b = np.asarray(y[index])
                
#                 print("x:", a.astype(int))
#                 print("y:", b)

                for c in range(len(b)):
                    w_x =  w[c] @ a
                    num = np.exp(w_x)

                    den = 0
                    for i in range(len(b)):
                        w_x =  w[i] @ a
                        den += np.exp(w_x)

                    yh_c = num/den

                    y_c = b[c]
                    
                    # TODO: may change, see slide 27 of logistic slideshow
                    cost_c = np.dot(yh_c - y_c, a)
                
                    gradients[c] += cost_c

            return gradients
        
        w0 = []
        for c in range(len(y[0])):
            w0.append(np.zeros(len(x[0])))
            
        self.w = optimizer.run(gradient, x, y, w0)
        return self
    
    def predict(self, x):
        a = np.asarray(x)
        b = np.asarray(self.w)
        
#         if self.add_bias:
#             x = np.column_stack([x,np.ones(N)])

        yh=[]
        for i,x_c in enumerate(a):
            yh_x=[]
            for c in range(len(b)):
                w_x =  b[c] @ x_c
                num = np.exp(w_x)

                den = 0
                for i in range(len(b)):
                    w_x =  b[i] @ x_c
                    den += np.exp(w_x)

                yh_c = num/den
                yh_x.append(yh_c)
            yh.append(yh_x)
        return yh

In [12]:
def cost(yh, y):
    return y * np.log1p(np.exp(-yh)) + (1-yh) * np.log1p(np.exp(yh))

# TODO: grid-search to find lowest cost combination of model hyper-parameters

batch_size = 10
learning_rate = 0.01
momentum = 0.9

digits_cost = 0

for fold_index, fold in enumerate(xDigitsTrainingSets):
    gradientDescentModel = GradientDescent(batch_size, learning_rate, momentum)
    logisticRegressionModel = LogisticRegression(False)
    
    logisticRegressionModel.fit(fold, yDigitsTrainingSets[fold_index], gradientDescentModel)
    yh = logisticRegressionModel.predict(xDigitsValidationSets[fold_index])
    
    for sample_index, yh_x in enumerate(yh):
        c = np.argmax(yDigitsValidationSets[fold_index][sample_index])
        cst = cost(yh_x[c], yDigitsValidationSets[fold_index][sample_index][c])
        digits_cost += cst
        print(yh_x, yDigitsValidationSets[fold_index][sample_index])
        print(cst)

wine_cost = 0

for fold_index, fold in enumerate(xWineTrainingSets):
    gradientDescentModel = GradientDescent(batch_size, learning_rate, momentum)
    logisticRegressionModel = LogisticRegression(False)
    
    logisticRegressionModel.fit(fold, yWineTrainingSets[fold_index], gradientDescentModel)
    yh = logisticRegressionModel.predict(xWineValidationSets[fold_index])
    
    for sample_index, yh_x in enumerate(yh):
        c = np.argmax(yWineValidationSets[fold_index][sample_index])
        cst = cost(yh_x[c], yWineValidationSets[fold_index][sample_index][c])
        wine_cost += cst
        print(yh_x, yWineValidationSets[fold_index][sample_index])
        print(cst)

print("Model hyper-parameters:")
print("\tMini-batch size:", batch_size)
print("\tLearning rate:", learning_rate)
print("\tMomentum:", momentum)
print("Digits total cost:", digits_cost)
print("Wine total cost:", wine_cost)



#xWineTrainingSets = []
#yWineTrainingSets = []
#xWineValidationSets = []
#yWineValidationSets = []

Grad descent step 1
0 : [ 0.00000000e+00 -7.00000000e-03 -8.60000000e-02 -1.96000000e-01
 -2.78000000e-01 -2.09000000e-01 -8.90000000e-02 -1.60000000e-02
  0.00000000e+00 -5.20000000e-02 -1.38000000e-01 -2.44000000e-01
 -1.17000000e-01 -1.17000000e-01 -8.90000000e-02 -1.00000000e-02
  0.00000000e+00 -4.20000000e-02 -1.82000000e-01 -2.09000000e-01
 -9.20000000e-02 -5.00000000e-02 -2.00000000e-03 -4.00000000e-03
  0.00000000e+00 -8.00000000e-03 -1.44000000e-01 -2.15000000e-01
 -2.37000000e-01 -6.10000000e-02 -5.00000000e-03  0.00000000e+00
  0.00000000e+00 -2.20000000e-02 -8.30000000e-02 -2.82000000e-01
 -3.76000000e-01 -1.49000000e-01 -5.30000000e-02  0.00000000e+00
  0.00000000e+00 -1.70000000e-02 -4.70000000e-02 -2.63000000e-01
 -2.51000000e-01 -4.40000000e-02 -4.60000000e-02  0.00000000e+00
  0.00000000e+00  6.00000000e-03  3.99680289e-17 -1.81000000e-01
 -1.35000000e-01 -9.30000000e-02 -9.60000000e-02 -1.10000000e-02
  0.00000000e+00 -4.00000000e-03 -9.80000000e-02 -2.12000000e-01
 


1 : [ 0.00000000e+00 -1.02889670e-01 -8.17593895e-01 -4.78487591e-01
 -7.76460455e-01 -1.12609717e+00 -2.37655126e-01 -8.35835323e-03
  2.59150918e-05 -5.74561165e-01 -3.36724295e-01  1.27161893e-02
 -5.46431007e-02  1.36821363e-01 -3.94922003e-01 -7.91169823e-02
  4.40658014e-05  3.60648480e-03  5.84437294e-01 -3.30655021e-01
 -1.62687462e+00  6.63937519e-01  1.45997394e-01 -7.55726100e-02
  0.00000000e+00  4.31070830e-01  6.76680145e-01 -1.81366013e+00
 -2.52116736e+00 -1.39141229e-02  5.08381257e-01 -8.18168949e-03
  0.00000000e+00  6.76839949e-01  6.04946944e-01 -1.87899534e+00
 -2.33805397e+00 -5.68805362e-01  6.33184825e-01  0.00000000e+00
 -2.01141508e-05  3.24103017e-01  1.49195464e+00 -1.16254797e+00
 -1.09589998e+00  2.82858427e-01  2.75576053e-01 -2.05454387e-05
  4.78637099e-05 -6.53741823e-02  6.14623764e-01 -2.27042034e-01
  2.34932740e-03  4.91588785e-01 -4.05507170e-01 -3.70769370e-04
  0.00000000e+00 -9.37338064e-02 -1.02854526e+00 -2.92890041e-01
 -2.13741226e-02 -2.

0 : [ 0.00000000e+00 -6.76425438e-02 -4.78980530e-02  1.02183546e+00
  5.53723002e-01 -5.06167107e-01 -1.57586308e-01 -4.22593399e-03
  1.36389903e-05 -2.71077049e-01  1.11357365e+00  1.18227519e+00
  1.00846662e+00  1.31665635e+00 -9.36938685e-02 -4.17802996e-02
  2.31916229e-05  3.29546415e-01  1.69796311e+00  1.12508687e-01
 -1.02456592e+00  1.55940296e+00  5.76647414e-01 -3.97923399e-02
  0.00000000e+00  7.84452156e-01  1.66553324e+00 -1.09951415e+00
 -1.78231708e+00  6.86539472e-01  1.04256666e+00 -4.30598449e-03
  0.00000000e+00  1.06180463e+00  1.61192366e+00 -1.10004154e+00
 -1.59191707e+00  3.21444084e-01  1.08484684e+00  0.00000000e+00
 -8.56012074e-05  6.29650414e-01  2.26607454e+00 -6.55605179e-01
 -5.94554055e-01  1.27556595e+00  6.57540075e-01 -4.94977917e-03
  2.51904442e-05  9.38774071e-02  1.63696967e+00  9.73816538e-01
  1.12985474e+00  1.45906608e+00 -2.35215016e-01 -6.38049152e-03
  0.00000000e+00 -5.88377864e-02 -1.96400121e-01  1.12921541e+00
  1.00714163e+00 -3.1

[1.4758378666602187e-17, 2.1441650758165603e-12, 2.5586165172647567e-09, 1.324074286644029e-05, 0.0010898320785223683, 0.8527308905025275, 0.14616603411532297, 2.205393763949952e-31, 2.3156636272760005e-29, 3.647404480755481e-26] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
1.3862943611198906
[1.051905993804937e-17, 1.0511124740729565e-12, 3.289235162520803e-09, 1.2330616395291812e-05, 0.0006535136961073738, 0.8332503022259443, 0.16608385017126667, 1.2981205818487265e-30, 8.14181288015284e-29, 1.3745203313914371e-25] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
1.3862943611198906
[1.8320490104171723e-16, 2.0555931245321515e-11, 1.5514833399034207e-08, 1.1127456962363415e-05, 0.0011436911997888843, 0.9800407252074637, 0.018804440600395555, 5.06820214863796e-40, 1.3166244953983573e-38, 4.5505858469286863e-35] [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1.3862943611198904
[9.196370915343514e-24, 1.8816836971089212e-16, 2.898980024323401e-12, 1.4580477646631753e-07, 0.00

3 : [ 0.00000000e+00  2.38333783e-03  4.66437834e-01  9.52206652e-01
  8.06474451e-01  1.16033618e-01 -2.33809026e-02 -2.43862227e-03
  0.00000000e+00  2.25352537e-01  1.11065373e+00  8.52426951e-01
  6.98351289e-01  8.75736069e-01  3.39473575e-02 -9.15729532e-04
  0.00000000e+00  4.15801304e-01  1.06972744e+00  7.07065766e-02
  2.64451597e-02  8.99878701e-01  2.64572920e-01 -1.72218063e-05
 -6.36127898e-04  5.48759668e-01  8.75466266e-01 -1.00661625e-01
 -1.35378192e-01  5.80879549e-01  4.76523739e-01  0.00000000e+00
  0.00000000e+00  5.56423306e-01  8.74899662e-01 -9.06180262e-02
 -1.23441709e-01  5.51041863e-01  5.63651714e-01  0.00000000e+00
  0.00000000e+00  3.33781570e-01  1.01501754e+00 -6.69092265e-02
  1.43707170e-02  7.78964561e-01  4.95591741e-01 -1.34871272e-03
  0.00000000e+00  3.32393406e-02  1.07235137e+00  5.87270312e-01
  6.67860378e-01  1.00327608e+00  2.46466150e-01 -1.20269429e-03
  0.00000000e+00  2.58408907e-03  4.05283745e-01  1.04988965e+00
  9.36178644e-01  4.1

...

[1.520863469348181e-45, 3.332811775321965e-38, 1.0781328477495317e-31, 2.296358433997226e-25, 2.1036815399460593e-18, 3.4347677670251105e-13, 7.741503669600027e-07, 0.031210648453203736, 0.9687885773960858, 8.854589586824533e-32] [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1.3862943611198906
[3.354059975847252e-48, 3.0114751657808244e-40, 5.274437800995714e-32, 1.446387592172649e-25, 5.941833290528377e-20, 2.00380595437287e-14, 9.160832094368626e-07, 0.037782237829157454, 0.962216846087613, 4.0739784145854074e-35] [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1.3862943611198906
[3.951176121123587e-43, 2.7954731882801663e-36, 9.021637271106986e-30, 1.975419869837265e-23, 7.619517380740939e-18, 9.224651189951112e-13, 1.9339000428234166e-06, 0.05180758758007098, 0.9481904785189638, 1.8621633054667806e-31] [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1.3862943611198906
[3.744714331172867e-46, 1.863099711524777e-38, 3.0765378288988045e-32, 6.020074187421028e-27, 1.64046710

[3.839588208817798e-53, 1.1231806366229816e-43, 7.393116257979214e-36, 3.1264926740663445e-29, 4.659728546127715e-22, 6.923582846549794e-16, 7.263312665478556e-08, 0.025745932763788056, 0.9742539946030846, 1.0744146667152735e-38] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
0.35357961501876956
[1.3949900886440454e-51, 1.3249257593987595e-42, 6.061456486572294e-35, 3.461823937580664e-28, 5.1945999445046236e-21, 2.7028680333870015e-15, 7.127512826448036e-08, 0.022782477689298464, 0.9772174510355706, 1.5781161782664707e-36] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
0.3489812257503927
[9.734687100359703e-48, 9.238982621659906e-40, 1.1381878480161468e-33, 2.1327159942862606e-28, 4.675480487129277e-20, 4.129566014329075e-15, 8.404426516203409e-07, 0.03845557616216051, 0.9615435833951836, 5.591608181984875e-35] [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1.3862943611198906
[1.4979915679053867e-45, 1.6894591582171927e-38, 7.554408749548525e-32, 3.573158788978129e-25, 7.45

0 : [-0.25667805 -0.21972047 -0.24311186 -0.34849525 -0.17450867 -0.06076851
  0.05852717 -0.31964899 -0.07499467 -0.10651703 -0.11222047 -0.09664
  0.10334853]

1 : [-0.2506738  -0.2166387  -0.23811489 -0.34208844 -0.17031477 -0.05755096
  0.06008119 -0.31429313 -0.07155673 -0.10609133 -0.10778085 -0.09142874
  0.10273357]

2 : [-0.25009129 -0.2141045  -0.23696247 -0.33969247 -0.17008857 -0.05913948
  0.05698827 -0.31164983 -0.07288288 -0.1040966  -0.10914491 -0.09368187
  0.10102895]

Grad descent step 18
0 : [-0.24302881 -0.22112356 -0.23176773 -0.34751416 -0.16201754 -0.04548036
  0.07351258 -0.31679474 -0.0659174  -0.10073502 -0.10017498 -0.07661466
  0.12467854]

1 : [-0.23981002 -0.21963992 -0.22861448 -0.34253136 -0.15999859 -0.04382609
  0.07426884 -0.31269139 -0.06399074 -0.10069013 -0.09742934 -0.07410358
  0.12267165]

2 : [-0.2393895  -0.21645478 -0.22814377 -0.34052158 -0.1599265  -0.04604252
  0.07053012 -0.31061487 -0.06544478 -0.09925835 -0.09929354 -0.07696492
  0.120

1.12264682959737
[0.34146711435441307, 0.3334188082465569, 0.3251140773990299] [0.0, 1.0, 0.0]
1.1226543533688156
[0.34088470337938076, 0.33330483654298426, 0.32581046007763503] [0.0, 1.0, 0.0]
1.122757236789477
[0.3414596851111982, 0.33334118534028784, 0.32519912954851393] [0.0, 1.0, 0.0]
1.1227244254309312
[0.3410598737358026, 0.3332603766697205, 0.3256797495944768] [0.0, 1.0, 0.0]
1.1227973684950108
[0.34118742610464875, 0.33337331066284326, 0.325439263232508] [0.0, 1.0, 0.0]
1.1226954256858825
[0.34044274992551254, 0.3332624393508873, 0.32629481072360017] [0.0, 1.0, 0.0]
1.1227955066490347
[0.3405641305896382, 0.3332431819587262, 0.32619268745163565] [0.0, 1.0, 0.0]
1.1228128888985134
[0.3402489603416236, 0.3332088366910516, 0.3265422029673248] [0.0, 1.0, 0.0]
1.1228438891782138
[0.3415176748438973, 0.3334373798780375, 0.32504494527806516] [0.0, 1.0, 0.0]
1.1226375876321906
[0.3415690787104311, 0.3334274315235876, 0.3250034897659813] [0.0, 1.0, 0.0]
1.1226465686469216
[0.3431157077


1 : [0.21721417 0.07704717 0.1740811  0.09205708 0.15742341 0.19677516
 0.17872062 0.07064402 0.13695577 0.12854142 0.12838604 0.2007821
 0.22115315]

2 : [0.20834906 0.07379116 0.16670191 0.08622518 0.15095504 0.1896095
 0.1727356  0.06614449 0.13160329 0.12492999 0.12211521 0.19268003
 0.21515602]

Grad descent step 7
0 : [0.23047637 0.07566386 0.18122581 0.08768491 0.16330687 0.21588012
 0.20092434 0.06558555 0.14872922 0.13492114 0.13725476 0.21874554
 0.24708629]

1 : [0.22934252 0.07461712 0.18069569 0.08844223 0.16274035 0.21443756
 0.19941341 0.06593964 0.14781709 0.13282886 0.13780368 0.21821039
 0.24404378]

2 : [0.22412527 0.07412406 0.17651613 0.08595379 0.15913726 0.2093802
 0.19450242 0.06445772 0.14434771 0.13151227 0.13326528 0.21220638
 0.23947309]

Grad descent step 8
0 : [0.22933974 0.07174996 0.17837029 0.08095327 0.16334923 0.21685536
 0.20947341 0.05369319 0.15090681 0.14114982 0.1365675  0.2225318
 0.26379287]

1 : [0.23342841 0.07323778 0.18201561 0.08552027 0.